In [1]:
import webuiapi
from PIL import Image, PngImagePlugin
import os

In [2]:
# 初始化api
api = webuiapi.WebUIApi()

url = "127.0.0.1"
# create API client with custom host, port
api = webuiapi.WebUIApi(host=url, port=7860)

In [3]:
# 各种生成图片的方法
# 文本生成图片
def text2img(api,prompt,style):
    result = api.txt2img(prompt=prompt,
                         seed=-1,
                         styles=style,
                         cfg_scale=7,
                         sampler_index='DDIM',
                         steps=30,
                         enable_hr=True,
                         hr_scale=1,
                         hr_upscaler=webuiapi.HiResUpscaler.Latent,
                         hr_second_pass_steps=20,
                         hr_resize_x=512,
                         hr_resize_y=768,
                         denoising_strength=0.4,
                         )
    return result

# 保存图片
def saveImages(result, imageDir,imageName,controlnet=False):
    imageNumber = len(result.images)
    # 如果使用了controlnet，那么最后一张图片是controlnet的结果，不需要保存
    if controlnet:
        imageNumber -= 2
    for i in range(imageNumber):
        image = result.images[i]
        pnginfo = PngImagePlugin.PngInfo()
        pnginfo.add_text("parameters", str(result.parameters))
        imagePath = imageDir + "/" + imageName + f"_{i}" + ".png"
        image.save(imagePath, pnginfo=pnginfo)
        
# 图生图
def img2img(api, prompt,imagePath):
    image = Image.open(imagePath)
    result = api.img2img(images=[image], 
                         prompt=prompt,
                         seed=-1, 
                         cfg_scale=6.5, 
                         denoising_strength=0.6
                         )
    return result

# 内画inpaint
def imgInpaint(api, prompt,negative_prompt,imagePath, maskPath):
    image = Image.open(imagePath)
    mask = Image.open(maskPath)
    result = api.img2img(images=[image],
                         mask_image=mask,
                         inpainting_fill=1,
                         prompt=prompt,
                         negative_prompt=negative_prompt,
                         seed=-1,
                         cfg_scale=5.0,
                         denoising_strength=0.7)
    return result

# 放大图片
def scaleImage(api, imagePath, scale):
    image = Image.open(imagePath)
    result = api.extra_single_image(image=image,
                                    upscaler_1=webuiapi.Upscaler.ESRGAN_4x,
                                    upscaling_resize=scale)
    return result

# 批量放大图片
def scaleImages(api, imageDir, scale):
    images = []
    # 遍历 imageDir,读取其中的png文件，放入images列表
    for file in os.listdir(imageDir):
        if file.endswith(".png"):
            image = Image.open(imageDir + "/" + file)
            images.append(image)

    result = api.extra_batch_images(images=images,
                                    upscaler_1=webuiapi.Upscaler.ESRGAN_4x,
                                    upscaling_resize=scale)
    return result

def changeImageSize(width, height):
    minSize = min(width, height)

    # 缩放宽高，使得最小的宽高为 512
    if minSize > 512:
        scale = minSize / 512
        widthScale = int(width / scale)
        heightScale = int(height / scale)
    else:
        scale = 512 / minSize
        widthScale = int(width * scale)
        heightScale = int(height * scale)

    # 将两个值都除以128，然后取整
    widthScale = int(widthScale/128) * 128
    heightScale = int(heightScale/128)*128

    return widthScale, heightScale

def checkImageExsit(imageDir, imageName):
    for file in os.listdir(imageDir):
        if file.startswith(imageName):
            return True
    return False

In [4]:
# 背景描述词
background = ["sea side","On the grass","In front of waterfall","beside a window"]

# 服装描述词
clothes = ["chinese dress",  "wedding dress", "nurse uniform","school uniform"]

# controlnet模块选择
controlnetUnits = ["canny","seg","reference"]

# 原图文件夹
originalDir = "E:\SharedDirectory\PoleDriver\AI参考"

# mask文件夹
maskDir = "mask"

# 风格描述词
real_stlye = "masterpiece, best quality, ultra high res, highres, best shadow, physics-based rendering, extremely delicate and beautiful,extremely detailed, amazing,shinning skin,"

# 负面描述词
negative_prompt = "easynegative"

In [5]:
# 使用图生图批量生成图片
# 设置图片保存路径
imageDir = "output"

# 设置图片描述词
description = "1 girl,look at viewer,"

# 设置批量生成图片的数量
batch_size = 4

# 设置使用的controlnet模块,选择 ["canny","seg","reference"] 中的一个
controlnetUnitName = controlnetUnits[2]

# 先用background和clothes各种组合生成提示词后缀，以及图片后缀名字。
# 因为window里文件命名不能有:，所以不可以直接用提示词后缀来命名图片
suffixs = []
suffixsName = []
for b in background:
    for c in clothes:
        suffixs.append(f"({b}:1.2)" + "," + f"({c}:1.5)")
        suffixsName.append(b + "_" + c)

# 遍历原图文件夹里所有的图片
for originalImag in os.listdir(originalDir):
        # 如果不是png文件，跳过
        if not originalImag.endswith(".png"):
            continue
        
        originalImagePath = originalDir + "/" + originalImag
        # 读取原图，以及其大小
        originalImage = Image.open(originalImagePath)
        originalImageWidth = originalImage.size[0]
        originalImageHeight = originalImage.size[1]
        
        # 读取原图片名字，去掉后缀.png
        originalImagName = originalImag.split(".")[0]
        
        if checkImageExsit(imageDir, originalImagName):
            print(f"图片{originalImagName}已经存在，跳过")
            continue
        
        print(f"原图名字：{originalImagName}。大小：{originalImageWidth}x{originalImageHeight}")
        width,height = changeImageSize(originalImageWidth, originalImageHeight)
        print(f"转换后大小：{width}x{height}")
        
        # 这下面是设置各种controlnet模块，如果混用需要设置weight，参数可以自己在webui上测试好再来填写，而且需要改写法
        if controlnetUnitName == "canny":
            # 设置controlnet模块，这里用的是canny边缘检测
            controlnetUnit = webuiapi.ControlNetUnit(input_image=originalImage, 
                                            module='canny', 
                                            model='control_v11p_sd15_canny [d14c016b]',
                                            resize_mode="Crop and Resize",
                                            threshold_a=100,
                                            threshold_b=200,
                                            )
        elif controlnetUnitName == "seg":
            # 设置controlnet模块，这里用的是碎片分割
            controlnetUnit = webuiapi.ControlNetUnit(input_image=originalImage,
                                            module='seg_ofade20k',
                                            model='control_v11p_sd15_seg [e1f51eb9]',
                                            resize_mode="Crop and Resize",
                                            )
        elif controlnetUnitName == "reference":
            # 设置controlnet模块，这里用的是图片参考
            controlnetUnit = webuiapi.ControlNetUnit(input_image=originalImage,
                                            module='reference_only',  
                                            resize_mode="Crop and Resize",
                                            )
        
        for i in range(len(suffixs)):
            suffix = suffixs[i]
            prompt = real_stlye + description + suffix
            suffixName = suffixsName[i]
            # 这里根据使用的controlnet模块，设置suffixName前面的类型名
            imageName = originalImagName + "_" + "reference" + suffixName
            
            print(f"使用的提示词是：{prompt}")
            result = api.txt2img(prompt=prompt, 
                                 negative_prompt=negative_prompt,  
                                 seed=-1,
                                 controlnet_units=[controlnetUnit],
                                 sampler_index='DPM++ 3M SDE Karras',
                                 steps=35,
                                 cfg_scale=6.5,
                                 enable_hr=True,                                 
                                 hr_resize_x=width,
                                 hr_resize_y=height,
                                 denoising_strength=0.6,
                                 batch_size=batch_size,  
                                 )
            # 保存图片
            saveImages(result, imageDir,imageName,controlnet=True)     

    
    

原图名字：00002。大小：2400x4248
转换后大小：512x896
使用的提示词是：masterpiece, best quality, ultra high res, highres, best shadow, physics-based rendering, extremely delicate and beautiful,extremely detailed, amazing,shinning skin,1 girl,look at viewer,(sea side:1.2),(chinese dress:1.5)
